# Example 1
* Create table to count the number of lesions found in LAD/LCx/RCA for each processed patient
    * Split the results into STEMI versus OTHER MI groups

## Import dependencies

In [15]:
import pyodbc
import pandas as pd

## Connect to database

In [3]:
db_type = 'Microsoft Access Driver (*.mdb, *.accdb)'
db_path = r"C:\Users\Admin\Desktop\db_CONFIRM.accdb"
pyodbc_str = (
    r'DRIVER={{{DB_TYPE}}};'
    r'DBQ={DB_PATH};'
    ).format(DB_TYPE=db_type, DB_PATH=db_path)

In [4]:
conn = pyodbc.connect(pyodbc_str)

## Create SQL query

In [5]:
sql_query = '''
SELECT tblMCP.id_patient, tblMCP.id_vessel, tblConfirmICONIC.mi_type, tblConfirmPerLesion.lesion_worst
FROM tblConfirmICONIC, tblConfirmPerLesion INNER JOIN (tblConfirmCONFIRM INNER JOIN tblMCP ON tblConfirmCONFIRM.confirm_idc_str = tblMCP.id_patient) ON tblConfirmPerLesion.confirm_idc = tblConfirmCONFIRM.[﻿confirm_idc]
WHERE (((tblMCP.[id_vessel]) Like '%_dist') AND ((tblConfirmPerLesion.lesion_worst)=1));
'''

In [6]:
#load the access query into a pandas dataframe
df = pd.read_sql_query(sql_query, conn)
df.head(20)

,id_patient,id_vessel,mi_type,lesion_worst
0,10128,lad_dist,2.0,1
1,10128,lad_dist,2.0,1
2,10128,lad_dist,1.0,1
3,10128,lad_dist,2.0,1
4,10128,lad_dist,2.0,1
5,10128,lad_dist,2.0,1
6,10128,lad_dist,1.0,1
7,10128,lad_dist,2.0,1
8,10128,lad_dist,4.0,1
9,10128,lad_dist,2.0,1


In [7]:
# sorting by id_patient
df.sort_values("id_patient", inplace = True)
  
# dropping duplicte values for id_patient 
df.drop_duplicates(subset ="id_patient", inplace = True) 
df.head()

,id_patient,id_vessel,mi_type,lesion_worst
0,10128,lad_dist,2.0,1
5305,10255,lcx_dist,4.0,1
9051,10399,rca_dist,2.0,1
10919,10947,lad_dist,4.0,1
12478,11273,lcx_dist,NaN,1


## Extract vessel and MI specific dataframes

In [8]:
df_lad_stemi = df[(df['id_vessel'] == 'lad_dist') & (df['mi_type'] == 1)]
df_lad_other = df[(df['id_vessel'] == 'lad_dist') & (df['mi_type'] != 1)]
df_lad_other = df_lad_other.dropna()

df_lcx_stemi = df[(df['id_vessel'] == 'lcx_dist') & (df['mi_type'] == 1)]
df_lcx_other = df[(df['id_vessel'] == 'lcx_dist') & (df['mi_type'] != 1)]
df_lcx_other = df_lcx_other.dropna()

df_rca_stemi = df[(df['id_vessel'] == 'rca_dist') & (df['mi_type'] == 1)]
df_rca_other = df[(df['id_vessel'] == 'rca_dist') & (df['mi_type'] != 1)]
df_rca_other = df_rca_other.dropna()

In [9]:
num_lad_stemi = df_lad_stemi.shape[0]
num_lad_other = df_lad_other.shape[0]

num_lcx_stemi = df_lcx_stemi.shape[0]
num_lcx_other = df_lcx_other.shape[0]

num_rca_stemi = df_rca_stemi.shape[0]
num_rca_other = df_rca_other.shape[0]

## Create a table (dataframe)
* Create table to count the number of lesions found in LAD/LCx/RCA for each processed patient
    * Split the results into STEMI versus OTHER MI groups

In [10]:
d = {
    'lad_stemi': num_lad_stemi,
    'percent_lad_stemi': ((num_lad_stemi / (num_lad_stemi + num_lad_other)) * 100),
    'lad_other': num_lad_other,
    'percent_lad_other': ((num_lad_other / (num_lad_stemi + num_lad_other)) * 100),
    'lcx_stemi': num_lcx_stemi,
    'percent_lcx_stemi': ((num_lcx_stemi / (num_lcx_stemi + num_lcx_other)) * 100),
    'lcx_other': num_lcx_other,
    'percent_lcx_other': ((num_lcx_other / (num_lcx_stemi + num_lcx_other)) * 100),
    'rca_stemi': num_rca_stemi,
    'percent_rca_stemi': ((num_rca_stemi / (num_rca_stemi + num_rca_other)) * 100),
    'rca_other': num_rca_other,
    'percent_rca_other': ((num_rca_other / (num_rca_stemi + num_rca_other)) * 100)
}

In [11]:
table = pd.DataFrame(d, index=[0])
table.head()

,lad_stemi,percent_lad_stemi,lad_other,percent_lad_other,lcx_stemi,percent_lcx_stemi,lcx_other,percent_lcx_other,rca_stemi,percent_rca_stemi,rca_other,percent_rca_other
0,1,6.25,15,93.75,4,28.571429,10,71.428571,9,28.125,23,71.875
